In [ ]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

ori_csv_data = pd.read_csv(os.path.join('sets', '165818_CKD-origin.csv'), encoding='ISO-8859-1')
print("Data shape (rows, columns) =  ", ori_csv_data.shape)

#print(ori_csv_data.info())
print("Columns: " + str(list(ori_csv_data.columns)))

In [ ]:
###################################
# data process
###################################
#step 1: remove non label(dCKD) rows
dat_proc_1 = ori_csv_data[ori_csv_data['dCKD'].notna()]
#print("Data shape (rows, columns) =  ", res_dat_proc_1.shape)

data_labels = dat_proc_1.dCKD.values
print("TRUE rate: ", data_labels.sum()/len(data_labels))
print("TRUE count: ", list(dat_proc_1.dCKD).count(1))
print("FALSE count: ", list(dat_proc_1.dCKD).count(0))

#step 2: choose relevant columns
#dat_proc_2 = dat_proc_1[[ 'age', 'height', 'weight', 'rLen', 'rShort', 'rPT', 'rPTLPA', 'LE', 'rKME', 'rKUPE', 'rRPE ', 'lPT', 'lPGA', 'lTSPA', 'lSPA', 'lPSA', 'lCKE', 'lRUPE', 'lRPE', 'DSK', 'DLK', 'DSR', 'DLR']]
dat_proc_2 = dat_proc_1[[ 'age', 'height', 'weight', 'rLen', 'rPT', 'rPTLPA', 'rKME', 'rKUPE', 'rRPE ', 'lPT', 'lPGA', 'lTSPA', 'lPSA', 'lCKE', 'lRUPE', 'lRPE', 'DSK', 'DLK']]
#dat_proc_2 = dat_proc_1[[ 'age', 'height', 'weight', 'rLen', 'rShort', 'rPT', 'rPTLPA', 'LE', 'rKME', 'rKUPE', 'rRPE ', 'lPT', 'lPGA', 'lTSPA', 'lSPA', 'lPSA', 'lCKE', 'lRUPE', 'lRPE', 'DSK', 'DLK']]
#dat_proc_2 = dat_proc_1[[ 'age', 'height', 'weight', 'rLen', 'rShort', 'rPT', 'rPTLPA', 'LE', 'rKME', 'rKUPE', 'rRPE ', 'lPT', 'lPGA', 'lTSPA', 'lSPA', 'lPSA', 'lCKE', 'lRUPE', 'lRPE', 'DSR', 'DLR']]
#print("Data shape (rows, columns) =  ", dat_proc_2.shape)
#print(res_dat_proc_2.count())

#step3-6: replace illigal data to median 
dat_proc_3 = dat_proc_2.replace('#DIV/0!', 0)
dat_proc_4 = dat_proc_3.fillna(0)
dat_proc_5 = dat_proc_4.astype('float32')
dat_proc_6 = dat_proc_5.replace(0, dat_proc_5.median())
#print(dat_proc_6.head(10))

#step 7: normalization
# max-min normalization
data = (dat_proc_6 - dat_proc_6.min())/(dat_proc_6.max() - dat_proc_6.min())
#print(data.head(10))

# z-score normalization
data1 = (dat_proc_6 - dat_proc_6.mean())/(dat_proc_6.std())  
#print(data1.head(10))

# Split dataset to training and testing dataset
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(data, data_labels, test_size=0.1, random_state=100)
print("training data shape (rows, columns) =  ", train_data.shape)
print("TRUE rate: ", train_labels.sum()/len(train_labels))

print("testing data shape (rows, columns) =  ", test_data.shape)
print("TRUE rate: ", test_labels.sum()/len(test_labels))


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
y = train_labels
X = train_data
fig = plt.figure(1, figsize=(20, 14))
ax = Axes3D(fig)
X_reduced = PCA(n_components=3).fit_transform(X)

colors = ['navy', 'turquoise', 'darkorange']

#for color, i, target_name in zip(colors, ['red', 'green', 'blue'], target_names):
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y,
           cmap=plt.cm.Set1, edgecolor='k')
ax.set_title("PCA")
ax.set_xlabel("1st eigenvector")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("3rd eigenvector")
ax.w_zaxis.set_ticklabels([])

plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(train_data, train_labels)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = test_labels, clf.predict(test_data)
    print(classification_report(y_true, y_pred))
    print()


In [ ]:
# Split training dataset to 2 group
tr_data, ts_data, tr_labels, ts_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=100)
print("self training data shape (rows, columns) =  ", tr_data.shape)
print("self test data shape (rows, columns) =  ", ts_data.shape)

from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=2000)
clf.fit(tr_data, tr_labels)

print("Weight vector: " + str(clf.coef_))
print("Bias: " + str(clf.intercept_))
print("Prediction accuracy:", clf.score(ts_data, ts_labels))

print("Prediction test accuracy:", clf.score(test_data, test_labels))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Set the parameters by cross-validation
tuned_parameters = [{'C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000, 10000]}]

print("# Tuning hyper-parameters for accuracy")
clf = GridSearchCV(LinearSVC(), tuned_parameters, cv=10, scoring='accuracy')
clf.fit(train_data, train_labels)

print("Best parameters set found on development set:")
print(clf.best_params_)
print("Grid scores on development set:")

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

#train and test by using best parameters.
clf = LinearSVC(random_state=10, C=clf.best_params_['C'])
clf.fit(tr_data, tr_labels)

print("Weights: " + str(clf.coef_))
print("Bias: " + str(clf.intercept_))
print("Prediction accuracy:", clf.score(ts_data, ts_labels))

print("Prediction test accuracy:", clf.score(test_data, test_labels))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

#tr_data, ts_data, tr_labels, ts_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=10)
clf = DecisionTreeClassifier(criterion='entropy')

scores = cross_val_score(clf, train_data, train_labels,  cv=5, scoring='f1')
print('Scores max', scores.max())
print('Scores min', scores.min())
print('Scores mean', scores.mean())

clf.fit(tr_data, tr_labels)

print("Accuracy", clf.score(ts_data, ts_labels))
print("The depth of this tree is", clf.tree_.max_depth)

#graph = graphviz.Source(tree.export_graphviz(clf, out_file=None)) 
#graph.render('graphs/tweet_dt_gini')

clf.fit(train_data, train_labels)
#print("Prediction eval accuracy:", clf.score(eval_data, eval_labels))
print("Prediction test accuracy:", clf.score(test_data, test_labels))


In [ ]:
clf = DecisionTreeClassifier(criterion='gini')
clf.fit(tr_data, tr_labels)
#DecisionTreeClassifier()

print("Accuracy", clf.score(ts_data, ts_labels))
print("The depth of this tree is", clf.tree_.max_depth)

#graph = graphviz.Source(tree.export_graphviz(clf, out_file=None)) 
#graph.render('graphs/dt_entropy')

clf.fit(train_data, train_labels)
#DecisionTreeClassifier()
#print("Prediction eval accuracy:", clf.score(eval_data, eval_labels))
print("Prediction test accuracy:", clf.score(test_data, test_labels))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=None, random_state=0, n_estimators=100, criterion='gini') 

scores = cross_val_score(clf, train_data, train_labels,  cv=5, scoring='f1')
print('Scores max', scores.max())
print('Scores min', scores.min())
print('Scores mean', scores.mean())

clf.fit(train_data, train_labels) 
print(str(list(train_data.columns)))
print(clf.feature_importances_)

#print("Prediction eval accuracy:", clf.score(eval_data, eval_labels)) 
print("Prediction test accuracy:", clf.score(test_data, test_labels))

In [ ]:
from sklearn.linear_model import LogisticRegression

tr_data, ts_data, tr_labels, ts_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=1)
clf = LogisticRegression(penalty= 'l2')

#train and test on setting A
clf.fit(tr_data, tr_labels)
print('Prediction Accuracy:', clf.score(ts_data, ts_labels))

print(str(list(tr_data.columns)))
print(clf.coef_)
print("bias: " + str(clf.intercept_))

#train and test on setting B
#clf.fit(tr_data, tr_labels)

print('Prediction Accuracy:', clf.score(test_data, test_labels))

In [ ]:

tr_data, ts_data, tr_labels, ts_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=1)

tuned_parameters = [{'C': [1e-2, 0.1, 0.25, 0.5, 0.75, 1, 10, 100, 1000]}]

clf = GridSearchCV(LogisticRegression(penalty= 'l2'), tuned_parameters, cv=5, scoring='accuracy')
clf.fit(train_data, train_labels)

print("Best parameters set found on development set:")
print(clf.best_params_)
print("Grid scores on development set:")

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

#train and test by using best parameters.
clf = LogisticRegression(penalty= 'l2', C=clf.best_params_['C'])

clf.fit(tr_data, tr_labels)

print("Weights: " + str(clf.coef_))
print("Bias: " + str(clf.intercept_))
print("Prediction accuracy:", clf.score(ts_data, ts_labels))

print("Prediction test accuracy:", clf.score(test_data, test_labels))